In [1]:
include("slim_reader.jl")
include("Encoder.jl")
using Knet

┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = top-level scope at slim_reader.jl:6
└ @ Core /home/ec2-user/comp492/Text2Image-cemil-slim-dataset-reader/slim_reader.jl:6


In [108]:
BATCH_SIZE = 50
slim_data_dir = "/home/ec2-user/efs/slim-dataset"

"/home/ec2-user/efs/slim-dataset"

In [3]:
syn_train_files = Slim_Reader.get_train_filenames(slim_data_dir, true, false)
# Construct training dataset object for synthetic data
syn_train_data = Slim_Reader.make_dataset_object(syn_train_files, BATCH_SIZE)




Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.



2019-11-03 08:07:10.327070: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2019-11-03 08:07:10.351646: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300060000 Hz
2019-11-03 08:07:10.352132: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x6430100 executing computations on platform Host. Devices:
2019-11-03 08:07:10.352156: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-3
OMP: Info #156: KMP_AFFINITY: 4 available OS procs
OMP: Info #157: K

Dict{String,Any} with 4 entries:
  "sess"         => PyObject <tensorflow.python.client.session.Session object a…
  "filenames"    => PyObject <tf.Tensor 'Placeholder:0' shape=(?,) dtype=string>
  "iterator"     => PyObject <tensorflow.python.data.ops.iterator_ops.Iterator …
  "next_element" => (((PyObject <tf.Tensor 'IteratorGetNext:0' shape=(?, 9, ?, …

In [4]:
# Iterate through the dataset and get the next batch
syn_train_batch = Slim_Reader.get_next_batch(syn_train_data, KnetArray{Float32}, false, true);

In [5]:
aimages, acaptions, acameras = getdata(syn_train_batch)

(K32(50,10,128,128,3)[0.6039216⋯], ["There is a large purple box behind a purple cone. The box is behind the cone.  " "There is a large purple box to the left of a purple cone.  " … "There is a purple cone to the left of a large purple box.  " "There is a purple cone to the left of a large purple box.  "; "There is a peach torus close to a peach ico. The torus is to the right of the ico. There is a purple cone to the right of a peach torus. The cone is close to the torus. There is a purple cone to the right of a peach ico.  " "There is a peach ico behind a peach torus. The ico is behind the torus. There is a peach torus behind a purple cone. The torus is behind the cone. There is a peach ico behind a purple cone. The ico is behind the cone.  " … "There is a peach ico close to a peach torus. The ico is in front of the torus. There is a purple cone to the left of a peach torus. The cone is close to the torus. There is a purple cone to the left of a peach ico.  " "There is a peach torus t

In [73]:
# create a dictionary of known words
vocabid = Dict()
vocabid = Dict("" => 1) # both padding and whitespace KnetArray(zeros(EMBEDDING_SIZE))
vocab = reshape(KnetArray(zeros(EMBEDDING_SIZE)),(EMBEDDING_SIZE,1))

64×1 KnetArray{Float64,2}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [74]:
function getembed(vocabid, vocab, w)
    if haskey(vocabid, w)
        return vocab[:,vocabid[w]]
    else
        println("There is no such word in the vocab!!!!")
    end
end

getembed (generic function with 1 method)

In [75]:
function updatevocabs!(vocabid, vocab, w)
    if haskey(vocabid, w)
        println("This word already exists...")
        return vocabid, vocab
    else
        vocabid[w] = length(vocabid)+1
        vocab = hcat(vocab, KnetArray(Knet.xavier(EMBEDDING_SIZE)))
        if size(vocab,2) == length(vocabid)
            println("Vocab and vocabid are in order!")
            return vocabid, vocab
        else
            return nothing
        end
    end    
end

updatevocabs! (generic function with 1 method)

In [76]:
function arrange(batchofcaptions, vocabid, vocab)
    int_batchofcaptions = []
    for i in 1:BATCH_SIZE
        int_captions = []
        captions = batchofcaptions[i,:]
        for c in captions
            int_cap = []
            str = split(c, " ")
            for i in 1:MAX_LENGTH
               if i <= length(str)
                    vocabid, vocab = updatevocabs!(vocabid, vocab, str[i])
                    push!(int_cap, getembed(vocabid, vocab, str[i])) #KnetArray(Knet.xavier(EMBEDDING_SIZE))
                else
                    push!(int_cap, getembed(vocabid, vocab, "")) # padding, same embedding with whitespace
                end
            end
            push!(int_captions, hcat(int_cap...))
        end
        push!(int_batchofcaptions, int_captions)
    end
    return vocabid, vocab, int_batchofcaptions
end

arrange (generic function with 1 method)

In [77]:
vocabid, vocab, caption_embeds = arrange(acaptions, vocabid, vocab)

Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
This word already exists...
This word already exists...
Vocab and vocabid are in order!
Vocab and vocabid are in order!
This word already exists...
This word already exists...
This word already exists...
Vocab and vocabid are in order!
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
Vocab and vocabid are in order!
This word already exists...
Vocab and vocabid are in order!
Vocab and vocabid are in order!
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

...

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

(Dict("pink" => 40,"cone" => 15,"right" => 19,"capsule" => 38,"teal" => 48,"of" => 14,"purple" => 6,"cylinder" => 34,"right." => 43,"to" => 12…), K64(64,48)[0.0⋯], Any[Any[K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯]], Any[K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯]], Any[K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯], K64(64,60)[0.09047945819803041⋯], K6

In [81]:
caption_embeds

10-element Array{Any,1}:
 K64(64,60)[0.09047945819803041⋯]
 K64(64,60)[0.09047945819803041⋯]
 K64(64,60)[0.09047945819803041⋯]
 K64(64,60)[0.09047945819803041⋯]
 K64(64,60)[0.09047945819803041⋯]
 K64(64,60)[0.09047945819803041⋯]
 K64(64,60)[0.09047945819803041⋯]
 K64(64,60)[0.09047945819803041⋯]
 K64(64,60)[0.09047945819803041⋯]
 K64(64,60)[0.09047945819803041⋯]

In [104]:
function createconvinput(caption_embeds)
    o = []
    for i in 1:BATCH_SIZE
        push!(o, cat(transpose(caption_embeds[i])...,dims=3))
    end
    return o = cat(o..., dims=3)
end

createconvinput (generic function with 1 method)

In [107]:
input = createconvinput(caption_embeds)

60×64×500 KnetArray{Float64,3}:
[:, :, 1] =
  0.0904795    0.163768   -0.171862    …   0.169092   -0.0919657   0.0193924 
 -0.0549839   -0.163613    0.00922363     -0.0105116  -0.0590364   0.061987  
 -0.13242      0.0749011  -0.00849783     -0.0577142   0.12524     0.0154178 
  0.155582    -0.117281   -0.0257867       0.0418501  -0.0231363   0.00409569
 -0.0125281   -0.048547   -0.0224799       0.0657487  -0.0252788  -0.0931465 
 -0.0585174   -0.0452983  -0.174223    …  -0.141019    0.142827    0.159795  
 -0.00462311   0.174029    0.122373        0.137589   -0.113599    0.0767912 
 -0.13242      0.0749011  -0.00849783     -0.0577142   0.12524     0.0154178 
 -0.0125281   -0.048547   -0.0224799       0.0657487  -0.0252788  -0.0931465 
  0.00743954  -0.0804394   0.165169       -0.0896698   0.0821539  -0.0292872 
 -0.0130666   -0.0617274  -0.0132451   …   0.0717476  -0.0851381   0.0745177 
 -0.0585174   -0.0452983  -0.174223       -0.141019    0.142827    0.159795  
 -0.0549839   -0.163

In [223]:
@doc conv4

```
conv4(w, x; kwargs...)
```

Execute convolutions or cross-correlations using filters specified with `w` over tensor `x`.

Currently KnetArray{Float32/64,4/5} and Array{Float32/64,4} are supported as `w` and `x`.  If `w` has dimensions `(W1,W2,...,I,O)` and `x` has dimensions `(X1,X2,...,I,N)`, the result `y` will have dimensions `(Y1,Y2,...,O,N)` where

```
Yi=1+floor((Xi+2*padding[i]-Wi)/stride[i])
```

Here `I` is the number of input channels, `O` is the number of output channels, `N` is the number of instances, and `Wi,Xi,Yi` are spatial dimensions.  `padding` and `stride` are keyword arguments that can be specified as a single number (in which case they apply to all dimensions), or an array/tuple with entries for each spatial dimension.

# Keywords

  * `padding=0`: the number of extra zeros implicitly concatenated at the start and at the end of each dimension.
  * `stride=1`: the number of elements to slide to reach the next filtering window.
  * `dilation=1`: dilation factor for each dimension.
  * `mode=0`: 0 for convolution and 1 for cross-correlation.
  * `alpha=1`: can be used to scale the result.
  * `handle`: handle to a previously created cuDNN context. Defaults to a Knet allocated handle.


In [166]:
filter = reshape(KnetArray(Knet.xavier(30*EMBEDDING_SIZE*32)), (1,30,EMBEDDING_SIZE,32)) # output size = embedding/input size
input = reshape(input, (1, MAX_LENGTH, EMBEDDING_SIZE, BATCH_SIZE*NUM_CAPTIONS_PER_SCENE))

1×60×64×500 KnetArray{Float64,4}:
[:, :, 1, 1] =
 0.0904795  -0.0549839  -0.13242  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 1] =
 0.163768  -0.163613  0.0749011  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3, 1] =
 -0.171862  0.00922363  -0.00849783  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

...

[:, :, 62, 1] =
 0.169092  -0.0105116  -0.0577142  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 63, 1] =
 -0.0919657  -0.0590364  0.12524  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 64, 1] =
 0.0193924  0.061987  0.0154178  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 1, 2] =
 0.0904795  -0.0549839  -0.13242  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 2] =
 0.163768  -0.163613  0.0749011  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3, 2] =
 -0.171862  0.00922363  -0.00849783  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

...

[:, :, 62, 2] =
 0.169092  -0.0105116  -0.0577142  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 63, 2] =
 -0.0919657  -0.0590364  0.12524  …  0.0  0.0  0.0  0.0  0.0 

In [167]:
@show y = conv4(filter, input, dilation=2)

y = conv4(filter, input, dilation=2) = K64(1,2,32,500)[-0.004344784949011223⋯]


1×2×32×500 KnetArray{Float64,4}:
[:, :, 1, 1] =
 -0.00434478  -0.000412513

[:, :, 2, 1] =
 0.00223635  0.00515137

[:, :, 3, 1] =
 -0.00104898  -0.0123837

...

[:, :, 30, 1] =
 -0.00324865  0.0137936

[:, :, 31, 1] =
 0.00400056  -0.010333

[:, :, 32, 1] =
 0.00394231  0.00188603

[:, :, 1, 2] =
 0.0038672  0.00452765

[:, :, 2, 2] =
 -0.00227756  0.0141548

[:, :, 3, 2] =
 0.00155014  -0.0062718

...

[:, :, 30, 2] =
 0.00457778  0.00296708

[:, :, 31, 2] =
 0.0117921  -0.00658569

[:, :, 32, 2] =
 0.0132619  0.00302149

[:, :, 1, 3] =
 0.000310047  -0.00347072

[:, :, 2, 3] =
 0.00302507  0.0100675

[:, :, 3, 3] =
 -0.0120596  -0.00224095

...

[:, :, 30, 3] =
 -0.00914533  0.00767003

[:, :, 31, 3] =
 -0.000699911  0.00665929

[:, :, 32, 3] =
 0.0045434  -0.00236434

...

[:, :, 1, 498] =
 0.000640726  -0.00893835

[:, :, 2, 498] =
 0.0150717  0.0187133

[:, :, 3, 498] =
 -0.0131346  0.00553394

...

[:, :, 30, 498] =
 -0.00934727  -0.00178922

[:, :, 31, 498] =
 0.00567684  0.000

In [151]:
function build_angles(tuplebatch)
    output = []
    for i in 1:BATCH_SIZE
        for j in 1:NUM_CAPTIONS_PER_SCENE
            sin_value = tuplebatch[i,j,3]/sqrt(tuplebatch[i,j,1]^2+tuplebatch[i,j,3]^2)
            cos_value = tuplebatch[i,j,1]/sqrt(tuplebatch[i,j,1]^2+tuplebatch[i,j,3]^2)
            push!(output, KnetArray([sin_value, cos_value])) 
        end
    end
    
    return cat(output..., dims=2)
end

build_angles (generic function with 1 method)

In [153]:
tuples = build_angles(acameras)

2×500 KnetArray{Float32,2}:
 0.420592   0.781365   0.421022   0.311719  …  0.673202  0.642129  0.287576
 0.90725   -0.624075  -0.90705   -0.950174     0.739459  0.766597  0.957758

In [132]:
struct Chain
    layers
    Chain(layers...) = new(layers)
end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)

In [133]:
struct Layer0; w; b; end
Layer0(ir::Int, ic::Int, o::Int) = Layer0(param(o,ir),param0(o, ic))
(l::Layer0)(x) = (l.w * x .+ l.b)

In [134]:
mlp1=Chain(Layer0(2, 500, 32)) # MLP1 dimensionality 32

Chain((Layer0(P(KnetArray{Float32,2}(32,2)), P(KnetArray{Float32,2}(32,500))),))

In [293]:
mlp2=Chain(Layer0(3860,2500), Layer0(2500,1000), Layer0(1000,256)) # MLP2 dimensionality 256

Chain((Layer0(P(KnetArray{Float32,2}(2500,3860)), P(KnetArray{Float32,1}(2500))), Layer0(P(KnetArray{Float32,2}(1000,2500)), P(KnetArray{Float32,1}(1000))), Layer0(P(KnetArray{Float32,2}(256,1000)), P(KnetArray{Float32,1}(256)))))

In [154]:
cameras_hat = mlp1(tuples)

32×500 KnetArray{Float32,2}:
  0.0527668   -0.180307    -0.165942    …   0.00601852   0.0767448 
 -0.0346578    0.237206     0.202349        0.0278574   -0.0677608 
 -0.100307     0.232505     0.228796       -0.0409668   -0.12978   
 -0.0247957   -0.129396    -0.0903169      -0.0601735   -0.00477875
 -0.111201     0.196224     0.205282       -0.0618957   -0.134885  
  0.11015     -0.135884    -0.157374    …   0.0769738    0.125065  
  0.0873348   -0.273534    -0.255088        0.0166279    0.123384  
  0.118502    -0.128308    -0.155254        0.0875984    0.131936  
  0.285103    -0.0243451   -0.150034        0.286905     0.275879  
  0.103241     0.166371     0.0833609       0.150811     0.0743053 
 -0.131511    -0.0453011    0.0247755   …  -0.147482    -0.118997  
  0.163516    -0.0573954   -0.120187        0.152918     0.164572  
  0.0594444   -0.137775    -0.13558         0.0242814    0.076909  
  ⋮                                     ⋱                          
  0.19348     -0.00

In [196]:
captions_hat = convert(KnetArray{Float32}, reshape(y, (2, 32, 500)))
cameras_hat = reshape(cameras_hat, (1, 32, 500))

1×32×500 KnetArray{Float32,3}:
[:, :, 1] =
 0.0527668  -0.0346578  -0.100307  …  -0.234358  -0.00721456  0.196902

[:, :, 2] =
 -0.180307  0.237206  0.232505  -0.129396  …  0.0273267  0.164417  0.0833489

[:, :, 3] =
 -0.165942  0.202349  0.228796  -0.0903169  …  0.129079  0.132539  -0.0248944

...

[:, :, 498] =
 -0.00143139  0.0375934  -0.0312999  …  -0.232329  0.0433249  0.228003

[:, :, 499] =
 0.00601852  0.0278574  -0.0409668  …  -0.23388  0.0366079  0.224972

[:, :, 500] =
 0.0767448  -0.0677608  -0.12978  …  -0.227844  -0.0309131  0.175898

In [308]:
# dimension düzenleeee 3x32x500 girsin 256 çıksın magicc
#mlp2(cat(captions_hat, cameras_hat, dims=1))

256-element KnetArray{Float32,1}:
  0.0009552671
  0.00499776  
 -0.009077348 
  0.0035285754
 -0.0039222194
  0.013600332 
  0.009764381 
  0.0028035284
 -0.00468146  
 -0.01868543  
  0.008617871 
  0.0011645677
 -0.0042066462
  ⋮           
  0.012515955 
 -0.005956823 
 -0.020680724 
 -0.009527057 
 -0.005908364 
 -0.0012455045
 -0.009382162 
  0.0040890644
 -0.017256409 
 -0.0013365897
  0.017339882 
  0.016975498 